# SNMP - Simple Network Management Protocol

## Install snmpd tools, agent and mibs

In [51]:
%%bash

# Install SNMP - Linux
sudo apt update
sudo apt install -y snmp snmpd snmp-mibs-downloader

Hit:1 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]
Get:4 https://dl.yarnpkg.com/debian stable InRelease [17.1 kB]
Hit:5 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:7 http://security.ubuntu.com/ubuntu focal-security InRelease


Hit:8 https://packagecloud.io/github/git-lfs/ubuntu focal InRelease
Fetched 145 kB in 1s (139 kB/s)
Reading package lists...
Building dependency tree...
Reading state information...
32 packages can be upgraded. Run 'apt list --upgradable' to see them.


Reading package lists...
Building dependency tree...
Reading state information...
snmp-mibs-downloader is already the newest version (1.2).
snmp is already the newest version (5.8+dfsg-2ubuntu2.9).
snmpd is already the newest version (5.8+dfsg-2ubuntu2.9).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.


## Configure agent configuration file (/etc/snmp/snmpd.conf) and snmp tools configuration file (/etc/snmp/snmp.conf)

In [53]:
%%bash

sudo mv /etc/snmp/snmpd.conf /etc/snmp/snmpd.conf.old

sudo tee /etc/snmp/snmpd.conf > /dev/null <<EOF
rocommunity public
rwcommunity private
EOF

sudo sed -i '/mibs :/s/^/#/' /etc/snmp/snmp.conf

## Start snmpd daemon (agent)

In [54]:
%%bash

sudo service snmpd restart
sudo service snmpd status

# Mac
# sudo launchctl unload /System/Library/LaunchDaemons/org.net-snmp.snmpd.plist
# sudo launchctl load -w /System/Library/LaunchDaemons/org.net-snmp.snmpd.plist

 * snmpd is running


## List all downloaded mibs

In [55]:
%%bash

ls -R /usr/share/snmp/mibs

/usr/share/snmp/mibs:
GNOME-SMI.txt
LM-SENSORS-MIB.txt
NET-SNMP-AGENT-MIB.txt
NET-SNMP-EXAMPLES-MIB.txt
NET-SNMP-EXTEND-MIB.txt
NET-SNMP-MIB.txt
NET-SNMP-MONITOR-MIB.txt
NET-SNMP-PASS-MIB.txt
NET-SNMP-PERIODIC-NOTIFY-MIB.txt
NET-SNMP-SYSTEM-MIB.txt
NET-SNMP-TC.txt
NET-SNMP-VACM-MIB.txt
UCD-DEMO-MIB.txt
UCD-DISKIO-MIB.txt
UCD-DLMOD-MIB.txt
UCD-IPFILTER-MIB.txt
UCD-IPFWACC-MIB.txt
UCD-SNMP-MIB-OLD.txt
UCD-SNMP-MIB.txt
iana
ietf


## Show RFC1213 object tree

In [56]:
%%bash

# Variables
MIB="/usr/share/snmp/mibs/ietf/RFC1213-MIB"

snmptranslate -m $MIB -Tp

+--iso(1)
   |
   +--org(3)
      |
      +--dod(6)
         |
         +--internet(1)
            |
            +--directory(1)
            |
            +--mgmt(2)
            |  |
            |  +--mib-2(1)
            |     |
            |     +--system(1)
            |     |  |
            |     |  +-- -R-- String    sysDescr(1)
            |     |  |        Textual Convention: DisplayString
            |     |  |        Size: 0..255
            |     |  +-- -R-- ObjID     sysObjectID(2)
            |     |  +-- -R-- TimeTicks sysUpTime(3)
            |     |  +-- -RW- String    sysContact(4)
            |     |  |        Textual Convention: DisplayString
            |     |  |        Size: 0..255
            |     |  +-- -RW- String    sysName(5)
            |     |  |        Textual Convention: DisplayString
            |     |  |        Size: 0..255
            |     |  +-- -RW- String    sysLocation(6)
            |     |  |        Textual Convention: DisplayString
           

## Show the description of a specific object

In [57]:
%%bash

# Variables
OBJECT="sysDescr"
MIB="/usr/share/snmp/mibs/ietf/RFC1213-MIB"

# Command to extract text from search_string to } excluding the }
sed -n "/$OBJECT OBJECT-TYPE/,/}/p" $MIB

sysDescr OBJECT-TYPE
    SYNTAX  DisplayString (SIZE (0..255))
    ACCESS  read-only
    STATUS  mandatory
    DESCRIPTION
            "A textual description of the entity.  This value
            should include the full name and version
            identification of the system's hardware type,
            software operating-system, and networking
            software.  It is mandatory that this only contain
            printable ASCII characters."
    ::= { system 1 }


## Using snmpget

In [58]:
%%bash

# snmpget -v <SNMP version> -c <community> <host> <obj1> <obj2> <obj3> 

snmpget -v 1 -c public localhost sysDescr.0 sysContact.0

SNMPv2-MIB::sysDescr.0 = STRING: Linux codespaces-581cab 6.5.0-1022-azure #23~22.04.1-Ubuntu SMP Thu May  9 17:59:24 UTC 2024 x86_64
SNMPv2-MIB::sysContact.0 = STRING: admin@localhost


## Using snmpset

In [59]:
%%bash

snmpset -v 1 -c private localhost sysContact.0 s admin@localhost
snmpget -v 1 -c public localhost sysContact.0

SNMPv2-MIB::sysContact.0 = STRING: admin@localhost
SNMPv2-MIB::sysContact.0 = STRING: admin@localhost


## Using snmpgetnext

In [60]:
%%bash

snmpgetnext -v 1 -c public localhost system interfaces

SNMPv2-MIB::sysDescr.0 = STRING: Linux codespaces-581cab 6.5.0-1022-azure #23~22.04.1-Ubuntu SMP Thu May  9 17:59:24 UTC 2024 x86_64
IF-MIB::ifNumber.0 = INTEGER: 3


## Using snmpwalk

In [ ]:
%%bash

# run on terminal: sudo tcpdump -i lo -n port 161 -v
snmpwalk -v 1 -c public localhost system

## Other commands

In [ ]:
%%bash

snmptable -v 1 -c public localhost ifTable

In [ ]:
%%bash

snmpdelta -v 1 -c public -Cs -CT localhost sysUpTime.0

In [ ]:
%%bash

killall snmpdelta

## Exercícios - SNMPv1

a) obter o endereço físico (MAC) da 2a. interface de rede da tabela ifTable

In [ ]:
%%bash

snmpget -v 1 -c public localhost ifPhysAddress.2

b) obter o número de mensagens ICMP enviadas e recebidas pelo elemento gerenciado

In [ ]:
%%bash

snmpget -v 1 -c public localhost icmpInMsgs.0 icmpOutMsgs.0

c) obter o número de requisições GET recebidos pelo agente no elemento gerenciado.

In [ ]:
%%bash

snmpget -v 1 -c public localhost snmpInGetRequests.0

d) modificar o nome do elemento gerenciado para "Gerencia"

In [ ]:
%%bash

snmpset -v 1 -c private localhost sysName.0 s "Gerencia"

e) modificar a identificação da localização do elemento gerenciado para "LabRedes"

In [ ]:
%%bash

snmpset -v 1 -c private localhost sysLocation.0 s "LabRedes"

f) obter a descrição da interface localizada na 1a. linha da tabela ifTable

In [ ]:
%%bash

snmpgetnext -v 1 -c public localhost ifDescr

g) obter o endereço IP, índice da interface na tabela ifTable, máscara de rede e endereço de broadcast da primeira entrada da tabela ipAddrEntry

In [ ]:
%%bash

snmpgetnext -v 1 -c public localhost ipAdEntIfIndex ipAdEntNetMask ipAdEntBcastAddr

h) obter o endereço local e porta local da primeira linha da tabela udpTable

In [ ]:
%%bash

snmpgetnext -v 1 -c public localhost udpLocalPort udpLocalAddress

## Using snmpbulkget

In [ ]:
%%bash

snmpbulkget -v2c -c public localhost -Cn2 -Cr3 system interfaces ifIndex ifDescr ifSpeed

In [ ]:
%%bash

snmpbulkget -v2c -c public localhost -Cn4 -Cr5 tcpInSegs tcpOutSegs tcpRetransSegs tcpInErrs tcpConnState tcpConnLocalAddress tcpConnLocalPort tcpConnRemAddress tcpConnRemPort

In [ ]:
%%bash

snmpbulkget -v2c -c public localhost -Cn0 -Cr4 udpLocalAddress udpLocalPort

## Comparing snmpwalk vs snmpbulkwalk



In [ ]:
%%bash

# run on terminal: sudo tcpdump -i lo -n port 161 -v
# snmpwalk -v 1 -c public localhost system

snmpbulkwalk -v 2c -c public localhost system

# Example using pass directive

## snmpd.conf

In [142]:
%%bash

# Add pass directive to snmpd.conf
sudo tee /etc/snmp/snmpd.conf > /dev/null <<EOF
rocommunity public
rwcommunity private

pass .1.3.6.1.3.1234.1 /usr/bin/python3 /tmp/agent.py
EOF

# Restart agent - Linux
sudo service snmpd restart
sudo service snmpd status

# Restart agent - Mac
# sudo launchctl unload /System/Library/LaunchDaemons/org.net-snmp.snmpd.plist
# sudo launchctl load -w /System/Library/LaunchDaemons/org.net-snmp.snmpd.plist

 * snmpd is running


## Agent code

In [141]:
%%bash 
pwd

/workspaces/AdvancedNetworks


In [151]:
%%writefile agent.py
#!/usr/bin/env python3

import sys
import datetime
import socket
import json

def get_revenue():
    f = open('logs.json')
    data = json.load(f)
    # print(data)
    f.close()
    return data['revenue']

def get_total_orders():
    f = open('logs.json')
    data = json.load(f)
    # print(data)
    f.close()
    return data['totalOrders']

def main():

    with open("agent.log", 'a') as file:
        file.write(' '.join(sys.argv)+"\n")

    # readJson()
    if len(sys.argv) != 3:
        print("Usage: agent.py <request-type> <MIB-oid>")
        return

    request_type = sys.argv[1]
    oid = sys.argv[2]

    if request_type == "-g":  # GET request # -s é set, -g 
        if oid == ".1.3.6.1.3.1234.1.1.0":
            print(".1.3.6.1.3.1234.1.1.0")
            print("integer")
            print(get_revenue())
        elif oid == ".1.3.6.1.3.1234.1.2.0":
            print(".1.3.6.1.3.1234.1.2.0")
            print("integer")
            print(get_total_orders())
        else:
            print("NONE")
    else:
        print("NONE")

if __name__ == "__main__":
    main()



Overwriting agent.py


In [ ]:
%%bash
sudo python /tmp/agent.py -g .1.3.6.1.3.1234.1.2.0

## Testing

In [152]:
%%bash

snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.1.0
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.2.0

SNMPv2-SMI::experimental.1234.1.1.0 = INTEGER: 0
SNMPv2-SMI::experimental.1234.1.2.0 = INTEGER: 0


In [78]:
%%writefile COFFEESHOP.txt

COFFEESHOP DEFINITIONS ::= BEGIN

IMPORTS
    MODULE-IDENTITY, OBJECT-TYPE,
    Integer32, experimental
        FROM SNMPv2-SMI
    DisplayString
        FROM SNMPv2-TC
        ;

coffee MODULE-IDENTITY
     LAST-UPDATED "9803231700Z"
     ORGANIZATION "The best coffeeshop in the world"

     CONTACT-INFO
            "        Gabriela Zorzo,
                     Lucas Andreotti,
                     Joana Figueredo
             "
     DESCRIPTION
            "The MIB Module for the best coffeeshop."
    ::= { experimental 1234 }

items OBJECT IDENTIFIER ::= { coffee 1 }

revenue OBJECT-TYPE
     SYNTAX  Integer32
     MAX-ACCESS read-only
     STATUS current
     DESCRIPTION
             "The total revenue of the coffeeshop."
     ::= { items 1 }


totalOrders OBJECT-TYPE
     SYNTAX Integer32
     MAX-ACCESS read-only
     STATUS current
     DESCRIPTION
        "The total number of orders placed in the coffeeshop."
    ::= { items 2 }

END


Overwriting COFFEESHOP.txt


In [82]:
%%bash

snmpget -v2c -c public -M +. -m +COFFEESHOP localhost revenue.0
snmpget -v2c -c public -M +. -m +COFFEESHOP localhost totalOrders.0

COFFEESHOP::revenue.0 = No Such Instance currently exists at this OID
COFFEESHOP::totalOrders.0 = No Such Instance currently exists at this OID


In [144]:
# %%write

from random import randrange
import time
import json

# Funcionários
employees = ["Gabriela","Joana","Lucas"]

# Items
items = ["coffee","tea","soda","muffin","sandwich","pie"]

# Número items
itemsNumber = [0,0,0,0,0,0]

# Tabela de preços
priceTable = {"coffee" : 8, "tea" : 5, "soda" : 6, "muffin" : 7, "sandwich" : 15, "pie" : 12 }

# Número pedidos
orders = 0

# Total valor
totalValue = 0

# Status
status = "Open"


while(True):
    index = randrange(0,5)
    item = items[index]
    value = priceTable[item]

    orders += 1
    totalValue += value
    itemsNumber[index] += 1

    log = {
        "revenue" : totalValue,
        "totalOrders" : orders,
        items[0] : itemsNumber[0],
        items[1] : itemsNumber[1],
        items[2] : itemsNumber[2],
        items[3] : itemsNumber[3],
        items[4] : itemsNumber[4],
        items[5] : itemsNumber[5],
        "employees" : employees,
        "priceTable" : priceTable,
        "status" : status
    }

    print(log)
   
    json_object = json.dumps(log, indent=4)
    
    with open("/tmp/logs.json", "w") as outfile:
        outfile.write(json_object)

    time.sleep(3)

{'revenue': 15, 'totalOrders': 1, 'coffee': 0, 'tea': 0, 'soda': 0, 'muffin': 0, 'sandwich': 1, 'pie': 0, 'employees': ['Gabriela', 'Joana', 'Lucas'], 'priceTable': {'coffee': 8, 'tea': 5, 'soda': 6, 'muffin': 7, 'sandwich': 15, 'pie': 12}, 'status': 'Open'}
{'revenue': 20, 'totalOrders': 2, 'coffee': 0, 'tea': 1, 'soda': 0, 'muffin': 0, 'sandwich': 1, 'pie': 0, 'employees': ['Gabriela', 'Joana', 'Lucas'], 'priceTable': {'coffee': 8, 'tea': 5, 'soda': 6, 'muffin': 7, 'sandwich': 15, 'pie': 12}, 'status': 'Open'}
{'revenue': 28, 'totalOrders': 3, 'coffee': 1, 'tea': 1, 'soda': 0, 'muffin': 0, 'sandwich': 1, 'pie': 0, 'employees': ['Gabriela', 'Joana', 'Lucas'], 'priceTable': {'coffee': 8, 'tea': 5, 'soda': 6, 'muffin': 7, 'sandwich': 15, 'pie': 12}, 'status': 'Open'}
{'revenue': 43, 'totalOrders': 4, 'coffee': 1, 'tea': 1, 'soda': 0, 'muffin': 0, 'sandwich': 2, 'pie': 0, 'employees': ['Gabriela', 'Joana', 'Lucas'], 'priceTable': {'coffee': 8, 'tea': 5, 'soda': 6, 'muffin': 7, 'sandwich'

KeyboardInterrupt: 